# The Viterbi Algorithm for decoding under the bigram model.

In [ ]:
# Import the required libraries
import operator
import numpy as np

In [ ]:
# arr stores all the bigram values line by line but it is still in string format
file_name = "" # Your file path that contains the bigram and its probabilities line by line in text format

# Example:
# '<start>' '<start>' 2.1782695826435478e-05
# '<start>' 'T' 0.20767622200923586
# '<start>' 'h' 0.003027794719874532

arr = np.loadtxt(file_name,dtype = "str", delimiter = "\n") # load the file into an array using np.loadtxt
bi_g = {} # bi_g contains the bi_gram model in a dictionary form
for tup in arr:
  bi_g[(tup.split()[0], tup.split()[1])] = float(tup.split()[2])
bi_g

In [ ]:
# A file that contains each line as a sentence and each character is separated by a space. Some of the characters are labeled as '<mask>' which we need to
# predict under the bi-gram model that maximizes the probability of the sentence.
file_name_masked = "" # Path of the file that contains the masked sentences line by line

# Example: <start> A <s> r e <mask> o l v i n g <s> f u n d <eos>
# <s> denotes a space character <start> denotes the beginning or start of the sentence and <eos> denotes the end of sentence.

arr_sent = np.loadtxt(file_name_masked, dtype = "str", delimiter = "\n")

In [ ]:
d_bi = {} 
# d_bi contains the count of each character that appears in the bi_gram, this is helpful is accessing the bi-gram characters
# with O(1) time since it is a dictionary

for k,v in bi_g.items():
  if k[0] not in d_bi:
    d_bi[k[0]] = 1
  else:
    d_bi[k[0]] += 1

In [ ]:
# The Viterbi algorithm function which takes in the sentence and bi_gram as parameters and returns the decoded sequence.

def Viterbi(sent, bi_gram):
  curr = ''
  mask_list_dict = []
  
  for ch in sent:
    if ch == '<start>':
      curr = '<start>'
      continue

    if ch != '<mask>':
      di_sub = {}
      if not mask_list_dict:
        di_sub[(curr, ch)] = np.log2(bi_gram[(curr, ch)])
        mask_list_dict.append(di_sub)
        curr = ch
        continue
    else:
      di_sub = {}
      if not mask_list_dict:
        for k,v in bi_gram.items():
          if k[0] == curr:
            di_sub[k] = np.log2(v)
        mask_list_dict.append(di_sub)
        curr = ch
        continue

    if curr != '<mask>':
      if ch == '<mask>':
        mask_vec = {}
      
        for k,v in bi_gram.items():
          if k[0] == curr and k[1] != "<start>":
            for km, vm in mask_list_dict[len(mask_list_dict)-1].items():
              if km[1] == curr:
                mask_vec[k] = (np.log2(v)) + vm
        mask_list_dict.append(mask_vec)
        curr = ch
        continue
      else:
        local_prob = bi_gram[(curr, ch)]
        di_sub = {}
        for km, vm in mask_list_dict[len(mask_list_dict)-1].items():
          if km[1] == curr:
            di_sub[(curr, ch)] = (np.log2(local_prob)) + vm 
        mask_list_dict.append(di_sub)
        curr = ch
        continue
      
    else:
      if ch != '<mask>':
        m_vec_sub = {}
        mask_vec_1 = {}
        
        for key,val in mask_list_dict[len(mask_list_dict)-1].items():
          m_vec_sub[(key[1],ch)] = (np.log2(bi_gram[(key[1],ch)])) + val
        max_key = max(m_vec_sub.items(), key=operator.itemgetter(1))[0]
        max_val = max(m_vec_sub.items(), key=operator.itemgetter(1))[1]
        mask_vec_1 = {max_key: max_val}

        mask_list_dict.append(mask_vec_1)
        curr = ch
        continue

      else:
          mask_vec = {}
          

          for ke, va in d_bi.items():
            di = {}
            for k,v in mask_list_dict[len(mask_list_dict)-1].items():
                di[(k[1], ke)] = (np.log2(bi_gram[(k[1], ke)])) + v
            max_k = max(di.items(), key=operator.itemgetter(1))[0]
            max_v = max(di.items(), key=operator.itemgetter(1))[1]
            mask_vec[max_k] = max_v

          mask_list_dict.append(mask_vec)
          curr = ch
          continue

  li_v3 = mask_list_dict
  unmasked_v3 = []
  curr_val = ""

  for d in reversed(li_v3):
    if list(d.keys())[0][1] == "<eos>":
      unmasked_v3.append(list(d.keys())[0][1])
      unmasked_v3.append(list(d.keys())[0][0])
      curr_val = list(d.keys())[0][0]
    else:
      ch = list(d)[0][0]
      unmasked_v3.append(ch)
  s = ""
  for ch in reversed(unmasked_v3):
    s += ch + " "
  return s

In [ ]:
# This is the code to store each of the decoded sentences in a list
decoded = []
for sent in arr_sent:
  test_st = sent.split()
  decoded.append(Viterbi(test_st, bi_g))

In [ ]:
# Coverting it into a text file
path = "" # The path where you want to share your decoded sentences text file.
with open('/content/drive/MyDrive/decoded_viterbi_15pt.txt','w') as writefile:
  for dec in decoded:
    writefile.write(dec + "\n")